sensitivity analysis - refutation

In [ ]:
from dowhy import CausalModel
from experiments import df_twins_no_missing_values, df_acs_no_missing_values

In [ ]:
import pandas as pd

def run_all_refutations(model, identified_estimand, estimate):
    refutation_results = []

    methods = [
        {"name": "random_common_cause", "label": "Add Random Common Cause"},
        {"name": "placebo_treatment_refuter", "label": "Placebo Treatment"},
        {"name": "dummy_outcome_refuter", "label": "Dummy Outcome"},
        {"name": "data_subset_refuter", "label": "Data Subsets Validation", "kwargs": {"subset_fraction": 0.9}},
        {"name": "bootstrap_refuter", "label": "Bootstrap Validation", "kwargs": {"num_simulations": 100}},
        {"name": "add_unobserved_common_cause", "label": "Add Unobserved Common Cause"}
    ]

    for m in methods:
        try:
            print(f"Running {m['label']}...")
            res = model.refute_estimate(
                identified_estimand,
                estimate,
                method_name=m['name'],
                **m.get("kwargs", {})
            )

            # --- תיקון השגיאה: טיפול במקרה שמוחזרת רשימה ---
            if isinstance(res, list):
                res = res[0]
            # ---------------------------------------------
            print(res.refutation_result)
            p_val = res.refutation_result.get('p_value')

            # בבדיקות אלו, p-value נמוך מ-0.05 מעיד על חוסר יציבות (FAIL)
            status = "PASS" if (p_val is not None and p_val >= 0.05) else "FAIL"

            refutation_results.append({
                "Method": m['label'],
                "p-value": round(p_val, 4) if isinstance(p_val, (int, float)) else p_val,
                "Result": status
            })
        except Exception as e:
            refutation_results.append({
                "Method": m['label'],
                "p-value": "Error",
                "Result": f"Error: {str(e)}"
            })

    return pd.DataFrame(refutation_results)


In [ ]:
model_twins=CausalModel(
        data = df_twins_no_missing_values,
        treatment='treatment',
        outcome='outcome',
        common_causes=df_twins_no_missing_values.columns.difference(["treatment", "outcome"]).tolist()
        )
identified_estimand_twins = model_twins.identify_effect(proceed_when_unidentifiable=True)
estimate_twins = model_twins.estimate_effect(identified_estimand_twins,method_name="backdoor.linear_regression")

run_all_refutations(model_twins, identified_estimand_twins, estimate_twins)

In [ ]:
model_acs=CausalModel(
        data = df_acs_no_missing_values,
        treatment='treatment',
        outcome='outcome',
        common_causes=df_acs_no_missing_values.columns.difference(["treatment", "outcome"]).tolist()
        )
identified_estimand_acs = model_acs.identify_effect(proceed_when_unidentifiable=True)
estimate_acs = model_acs.estimate_effect(identified_estimand_acs,method_name="backdoor.linear_regression")

run_all_refutations(model_acs, identified_estimand_acs, estimate_acs)

In [ ]:
refutation = model_twins.refute_estimate(
    identified_estimand_twins,
    estimate_twins,
    method_name="add_unobserved_common_cause",
    confounders_effect_on_treatment="linear",
    confounders_effect_on_outcome="linear"
)
print(refutation)

refutation = model_acs.refute_estimate(
    identified_estimand_acs,
    estimate_acs,
    method_name="add_unobserved_common_cause",
    confounders_effect_on_treatment="linear",
    confounders_effect_on_outcome="linear"
)
print(refutation)